# Import modules

In [23]:
# # Import libraries
# !pip install py_stringsimjoin
# !pip install py_stringmatching
import json, copy, os, time
import pandas as pd
import py_stringmatching as sm
import py_stringsimjoin as ssj
from py_stringmatching.tokenizer.delimiter_tokenizer import DelimiterTokenizer
from py_stringmatching.tokenizer.qgram_tokenizer import QgramTokenizer
from itertools import product

# Define measure and tokenizer

In [24]:
TOKENIZERS = {'SPACE_DELIMITER': DelimiterTokenizer(delim_set={' '}, return_set=True),
              '2_GRAM': QgramTokenizer(qval=2, padding=False, return_set=True),
              '3_GRAM': QgramTokenizer(qval=3, padding=False, return_set=True),
              '4_GRAM': QgramTokenizer(qval=4, padding=False, return_set=True),
              '5_GRAM': QgramTokenizer(qval=5, padding=False, return_set=True),
              '2_GRAM_BAG': QgramTokenizer(qval=2),
              '3_GRAM_BAG': QgramTokenizer(qval=3)
              }

# Define filters

In [25]:
FILTERS = {
    "OVERLAP_FILTER": ssj.OverlapFilter,
    "SIZE_FILTER": ssj.SizeFilter,
    "PREFIX_FILTER": ssj.PrefixFilter,
    "POSITION_FILTER": ssj.PositionFilter,
    "SUFFIX_FILTER": ssj.SuffixFilter
}

# Define string similarity functions

In [26]:
SIM_FUNC = {
    'COSINE': sm.Cosine().get_sim_score,
    'DICE': sm.Dice().get_sim_score,
    'JACCARD': sm.Jaccard().get_sim_score,
    'OVERLAP_COEFFICIENT': sm.OverlapCoefficient().get_sim_score,
    'LEVENSHTEIN': sm.Levenshtein().get_sim_score,
    'TF-IDF': sm.TfIdf,
}

# Define directory and load script function

In [27]:
DATA_PATH = os.sep.join([os.getcwd(), 'dataset'])
BENCHMARK_DIRECTORY = 'benchmark_results'

In [28]:
def load_scripts(scripts_file_name):
    with open(scripts_file_name, 'r') as js_file:
        scripts = json.load(js_file)['scripts']
    return scripts

# Apply each testing configuration for dataset

In [29]:
def test(script, l_table, r_table, idx_script, scripts_file_name):
    result = []
    total_info_obj = product(script['sim_funcs'], script['sim_measure_types'],
                            script['tokenizers'], script['scale_filters'],
                            script['thresholds'], script['n_jobs'])
    for sim_funcs, sim_measure_type, tokenizer, scale_filter, threshold, n_jobs in total_info_obj:
        if tokenizer in ["SPACE_DELIMITER"] and sim_measure_type != 'EDIT_DISTANCE':
            continue
        sim_func = SIM_FUNC[sim_funcs]
        tok = TOKENIZERS[tokenizer]
        if scale_filter == "OVERLAP_FILTER":
            s_filter = FILTERS[scale_filter](tok, overlap_size=1, comp_op='>=', allow_missing=False)
        else:
            s_filter = FILTERS[scale_filter](tok, sim_measure_type, threshold, allow_empty=True, allow_missing=False)
        sum_time = 0
        start_time = time.time()
        print(s_filter)
        candidate_set = s_filter.filter_tables(
            l_table, r_table,
            script['l_id_attr'], script['r_id_attr'],
            script['l_join_attr'], script['r_join_attr'],
            l_out_attrs=None, r_out_attrs=None,
            l_out_prefix='l_', r_out_prefix='r_',
            n_jobs=n_jobs, show_progress=False)
        output_table = ssj.apply_matcher(candidate_set,
                                        'l_' + script['l_id_attr'], 'r_' + script['r_id_attr'], l_table, r_table,
                                        script['l_id_attr'], script['r_id_attr'],
                                        script['l_join_attr'], script['r_join_attr'],
                                        tokenizer=tok, sim_function=sim_func, threshold=threshold,
                                        comp_op='>=', allow_missing=True,
                                        l_out_attrs=[script['l_join_attr']], r_out_attrs=[script['r_join_attr']],
                                        l_out_prefix='l_', r_out_prefix='r_',
                                        out_sim_score=True, n_jobs=n_jobs, show_progress=False)
        sum_time += (time.time() - start_time)
        cand_set_size = len(candidate_set)
        if not os.path.exists(BENCHMARK_DIRECTORY):
            os.makedirs(BENCHMARK_DIRECTORY)
        if not os.path.exists(os.sep.join([BENCHMARK_DIRECTORY, str(idx_script)])):
            os.makedirs(os.sep.join([BENCHMARK_DIRECTORY, str(idx_script)]))
        output_table.to_csv(os.sep.join([BENCHMARK_DIRECTORY, str(idx_script),
            scripts_file_name + sim_measure_type + '_' + tokenizer + str(threshold) + '_' + str(n_jobs) + scale_filter + '_' + sim_funcs + '_' + str(idx_script) + '.csv']))
        obj_res = {"idx_script" : idx_script, "table": copy.deepcopy(output_table), "time" : sum_time, "num_candidate": cand_set_size}
        result.append(obj_res)
    return result

In [30]:
def load_data_and_test(scripts_file_name):
    result = []
    scripts = load_scripts(scripts_file_name)
    for idx, script in enumerate(scripts):
        l_path = os.sep.join([DATA_PATH, *script['ltable']])
        r_path = os.sep.join([DATA_PATH, *script['rtable']])
        l_table = pd.read_csv(l_path, encoding=script['ltable_encoding'])
        r_table = pd.read_csv(r_path, encoding=script['rtable_encoding'])
        result.append(test(script, l_table, r_table, idx, scripts_file_name))
    return result, l_table, r_table
#result là mảng chiều: chiều thứ nhất tương ứng với chỉ số script trong list các script
#chiều thứ hai là dictionary ứng với từng lần đo trong mỗi script 

In [31]:
result, l_table, r_table = load_data_and_test("scripts_1.json")

c:\Users\Lenovo\anaconda3\lib\site-packages\py_stringsimjoin\utils\generic_helper.py:76: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only.
  projected_dataframe = dataframe[proj_attrs].dropna(0,
c:\Users\Lenovo\anaconda3\lib\site-packages\py_stringsimjoin\utils\generic_helper.py:76: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only.
  projected_dataframe = dataframe[proj_attrs].dropna(0,


c:\Users\Lenovo\anaconda3\lib\site-packages\py_stringsimjoin\utils\generic_helper.py:76: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only.
  projected_dataframe = dataframe[proj_attrs].dropna(0,
c:\Users\Lenovo\anaconda3\lib\site-packages\py_stringsimjoin\utils\generic_helper.py:76: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only.
  projected_dataframe = dataframe[proj_attrs].dropna(0,


c:\Users\Lenovo\anaconda3\lib\site-packages\py_stringsimjoin\utils\generic_helper.py:76: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only.
  projected_dataframe = dataframe[proj_attrs].dropna(0,
c:\Users\Lenovo\anaconda3\lib\site-packages\py_stringsimjoin\utils\generic_helper.py:76: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only.
  projected_dataframe = dataframe[proj_attrs].dropna(0,


c:\Users\Lenovo\anaconda3\lib\site-packages\py_stringsimjoin\utils\generic_helper.py:76: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only.
  projected_dataframe = dataframe[proj_attrs].dropna(0,
c:\Users\Lenovo\anaconda3\lib\site-packages\py_stringsimjoin\utils\generic_helper.py:76: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only.
  projected_dataframe = dataframe[proj_attrs].dropna(0,


c:\Users\Lenovo\anaconda3\lib\site-packages\py_stringsimjoin\utils\generic_helper.py:76: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only.
  projected_dataframe = dataframe[proj_attrs].dropna(0,
c:\Users\Lenovo\anaconda3\lib\site-packages\py_stringsimjoin\utils\generic_helper.py:76: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only.
  projected_dataframe = dataframe[proj_attrs].dropna(0,


# Show an example of joined result 

### Left table, right table before join

In [10]:
l_table

,id,title,authors,venue,year
0,journals/sigmod/Mackay99,Semantic Integration of Environmental Models for Application to Global Information Systems and D...,D. Scott Mackay,SIGMOD Record,1999
1,conf/vldb/PoosalaI96,Estimation of Query-Result Distribution and its Application in Parallel-Join Load Balancing,"Viswanath Poosala, Yannis E. Ioannidis",VLDB,1996
2,conf/vldb/PalpanasSCP02,Incremental Maintenance for Non-Distributive Aggregate Functions,"Themistoklis Palpanas, Richard Sidle, Hamid Pirahesh, Roberta Cochrane",VLDB,2002
3,conf/vldb/GardarinGT96,Cost-based Selection of Path Expression Processing Algorithms in Object-Oriented Databases,"Zhao-Hui Tang, Georges Gardarin, Jean-Robert Gruser",VLDB,1996
4,conf/vldb/HoelS95,Benchmarking Spatial Join Operations with Spatial Output,"Erik G. Hoel, Hanan Samet",VLDB,1995
...,...,...,...,...,...
2611,journals/tods/KarpSP03,A simple algorithm for finding frequent elements in streams and bags,"Scott Shenker, Christos H. Papadimitriou, Richard M. Karp",ACM Trans. Database Syst.,2003
2612,conf/vldb/LimWV03,SASH: A Self-Adaptive Histogram Set for Dynamically Changing Workloads,"Lipyeow Lim, Min Wang, Jeffrey Scott Vitter",VLDB,2003
2613,journals/tods/ChakrabartiKMP02,Locally adaptive dimensionality reduction for indexing large time series databases,"Kaushik Chakrabarti, Eamonn J. Keogh, Michael J. Pazzani, Sharad Mehrotra",ACM Trans. Database Syst.,2002
2614,journals/sigmod/Snodgrass01,Chair's Message,Richard T. Snodgrass,SIGMOD Record,2001


In [11]:
r_table

,id,title,authors,venue,year
0,304586,The WASA2 object-oriented workflow management system,"Gottfried Vossen, Mathias Weske",International Conference on Management of Data,1999
1,304587,A user-centered interface for querying distributed multimedia databases,"Isabel F. Cruz, Kimberly M. James",International Conference on Management of Data,1999
2,304589,"World Wide Database-integrating the Web, CORBA and databases","Athman Bouguettaya, Boualem Benatallah, Lily Hendra, James Beard, Kevin Smith, Mourad Quzzani",International Conference on Management of Data,1999
3,304590,XML-based information mediation with MIX,"Chaitan Baru, Amarnath Gupta, Bertram Lud&#228;scher, Richard Marciano, Yannis Papakonstantinou,...",International Conference on Management of Data,1999
4,304582,The CCUBE constraint object-oriented database system,"Alexander Brodsky, Victor E. Segal, Jia Chen, Paval A. Exarkhopoulo",International Conference on Management of Data,1999
...,...,...,...,...,...
2289,672977,Dual-Buffering Strategies in Object Bases,"Alfons Kemper, Donald Kossmann",Very Large Data Bases,1994
2290,950482,Guest editorial,"Philip A. Bernstein, Yannis Ioannidis, Raghu Ramakrishnan",The VLDB Journal &mdash; The International Journal on Very Large Data Bases,2003
2291,672980,GraphDB: Modeling and Querying Graphs in Databases,Ralf Hartmut G&#252;ting,Very Large Data Bases,1994
2292,945741,Review of The data warehouse toolkit: the complete guide to dimensional modeling (2nd edition) b...,Alexander A. Anisimov,ACM SIGMOD Record,2003


### Example of a joined-table using Position Filter, threshold 0.7

In [13]:
result[0][2]['table'] 

,_id,l_id,r_id,l_title,r_title,_sim_score
0,177,conf/sigmod/BouguettayaBH99,304589,"World Wide Database - Integrating the Web, CORBA, and Databases","World Wide Database-integrating the Web, CORBA and databases",0.811321
1,225,conf/sigmod/BaruGLMPVC99,304590,XML-Based Information Mediation with MIX,XML-based information mediation with MIX,0.710526
2,362,conf/sigmod/RoussopoulosKS99,304584,The Active MultiSync Controller of the Cubetree Storage Organization,The active MultiSync controller of the cubetree storage organization,0.750000
3,932,journals/sigmod/Danish98,306103,Building Database-driven Electronic Catalogs,Building database-driven electronic catalogs,0.744681
4,985,journals/sigmod/MeltzerC98,306105,XML and Electronic Commerce: Enabling the Network Economy,XML and electronic commerce: enabling the network economy,0.758621
...,...,...,...,...,...,...
365,152672,journals/vldb/AbbadiSW01,950482,Guest editorial,Guest editorial,1.000000
366,152673,journals/vldb/Atkinson00,950482,Guest editorial,Guest editorial,1.000000
367,152674,conf/vldb/Guting94,672980,GraphDB: Modeling and Querying Graphs in Databases,GraphDB: Modeling and Querying Graphs in Databases,1.000000
368,152702,journals/sigmod/Anisimov03,945741,Review of The data warehouse toolkit: the complete guide to dimensional modeling (2nd edition) b...,Review of The data warehouse toolkit: the complete guide to dimensional modeling (2nd edition) b...,0.992126


# Load data from DBLP-ACM_perfectMapping.csv

* The table below is the correct matching result (ground true table) that can be used for benchmark. 

In [35]:
import numpy as np
def caculate_true_positive(joined_table, true_label_table):
    joined_tuples = joined_table[["l_id", "r_id"]].to_records(index = False).astype(dtype=[('l_id', 'O'), ('r_id', 'O')])
    true_tuples = true_label_table[["l_id", "r_id"]].to_records(index = False).astype(dtype=[('l_id', 'O'), ('r_id', 'O')])
    return len(np.intersect1d(joined_tuples, true_tuples))
true_label_table = pd.read_csv("./dataset/library/DBLP-ACM_perfectMapping.csv", header = 0, encoding="ISO-8859-1", names = ['l_id', 'r_id'])
true_label_table

,l_id,r_id
0,conf/sigmod/SlivinskasJS01,375678
1,conf/sigmod/ChaudhuriDN01,375694
2,conf/sigmod/RinfretOO01,375669
3,conf/sigmod/BreunigKKS01,375672
4,conf/sigmod/JagadishJOT01,375687
...,...,...
2219,journals/sigmod/Scholl01,604275
2220,journals/sigmod/Rosneblatt94,190649
2221,journals/sigmod/Winslett02b,601871
2222,journals/sigmod/Labrinidis01,604283


# Encode result to .plk file and load file

In [ ]:
import pickle
pickle_obj = {'result': result, 'l_table' : l_table, 'r_table':r_table}
pickle.dump(pickle_obj, open("script_0.pkl", "wb"))

In [15]:
load_obj = pickle.load(open("script_0.pkl", "rb"))
temp_result = load_obj['result']

# Caculate metrics for result

We consider that result from load_data_and_test function is the prediction, and the correct matching from above table is the "ground true labels". 

In [33]:
def calculate_metrics(result, true_label_table):
    for i in range(0, len(result)):
        for obj_res in result[i]:
            obj_res["true_positive"] = caculate_true_positive(obj_res["table"], true_label_table)
            obj_res["false_positive"] = len(obj_res["table"]) - obj_res["true_positive"]
            obj_res["false_negative"] = len(true_label_table) - obj_res["true_positive"]
            obj_res["recall"] = obj_res["true_positive"] / (obj_res["true_positive"] + obj_res["false_negative"])
            obj_res["precision"] = obj_res["true_positive"] / (obj_res["true_positive"] + obj_res["false_positive"])
            obj_res["F1"] = 2 * obj_res["precision"] * obj_res["recall"] / (obj_res["precision"] + obj_res["recall"])

In [36]:
calculate_metrics(result, true_label_table)

# Show the caculated metrics from the scripts

In [26]:
for x in range(0, len(temp_result)):
    precision_list = []
    recall_list = []
    f1_list = []
    time_list = []
    num_candidate_list = []
    for y in range(0, len(temp_result[x])):
        precision_list.append(temp_result[x][y]["precision"])
        recall_list.append(temp_result[x][y]["recall"])
        f1_list.append(temp_result[x][y]["F1"])
        time_list.append(temp_result[x][y]["time"])
        num_candidate_list.append(temp_result[x][y]["num_candidate"])
    if x > 0:
        data_frame = pd.DataFrame({"precision": precision_list, "recall": recall_list, "f1": f1_list, "time": time_list})
    else: 
        data_frame = pd.DataFrame({"time": time_list, "num_candidate": num_candidate_list})
    print(data_frame)
    print("\n")


         time  num_candidate
0    3.833089           4515
1   38.655310        2806383
2    1.698787          55697
3  209.562787         860760


   precision    recall        f1       time
0   0.851468  0.665018  0.746781  23.681954
1   0.845070  0.458633  0.594579  15.376251
2   0.912451  0.421763  0.576876   5.057908


   precision    recall        f1      time
0   0.911005  0.428058  0.582441  1.246634
1   0.912366  0.421313  0.576438  0.927507
2   0.912745  0.418615  0.573983  0.930105




* The first table is comparision of performance of 4 filters: position, prefix, size, suffix. The similarity measure is Jaccard, threshold 0.7.


* The second table is comparision of time and accuracy metrics by the change of threshold 0.7 -> 0.8 -> 0.9.

* The third table is comparision of time and accuracy metrics by the using 3-gram, 4-gram, 5-gram tokenizers with Jaccard similarity measure, threhold 0.8. 

In [42]:
temp_result = result
for x in range(0, len(temp_result)):
    precision_list = []
    recall_list = []
    f1_list = []
    time_list = []
    num_candidate_list = []
    for y in range(0, len(temp_result[x])):
        precision_list.append(temp_result[x][y]["precision"])
        recall_list.append(temp_result[x][y]["recall"])
        f1_list.append(temp_result[x][y]["F1"])
        time_list.append(temp_result[x][y]["time"])
        num_candidate_list.append(temp_result[x][y]["num_candidate"])
        data_frame = pd.DataFrame({"precision": precision_list, "recall": recall_list, "f1": f1_list, "time": time_list, "num_candidate" : num_candidate_list})
    print(data_frame)
    print("\n")


   precision    recall        f1      time  num_candidate
0   0.792288  0.979317  0.875930  9.932187         391289
1   0.859917  0.935701  0.896210  6.164150         271802
2   0.851468  0.665018  0.746781  4.058825         152755
3   0.845070  0.458633  0.594579  2.023184          61095
4   0.912451  0.421763  0.576876  1.761235          12121


